<a href="https://colab.research.google.com/github/kazishuvo22/BigMart_Sales_Prediction/blob/master/BigMart_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>